In [1]:
#Importar
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#Acceso: client id, client secret
Client_ID = "bd39a5f17be2422090f8beab3ab83461"
Client_secret = "dacb9e8f9b1c4ad9be7dd105496bc81f"

In [3]:
# Autenticacion
auth_manager = SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
# Función para etraer todos los resultados
def extraer_datos_por_anios(sp, genres, anios, max_por_anio=1000):
    results = []
    artistas = set()  

    for anio in anios:
        print(f"\n Extrayendo datos del año {anio}")

        for genre in genres:
            print(f" Género: {genre}")

            offset = 0
            obtenidos = 0

            while obtenidos < max_por_anio:
                response = sp.search(
                    q=f'genre:{genre} year:{anio}',
                    type='track,album',
                    limit=50,
                    offset=offset
                )

                tracks = response['tracks']['items']
                albums = response['albums']['items']

                if not tracks and not albums:
                    break  

                # Canciones
                for item in tracks:
                    if obtenidos >= max_por_anio:
                        break

                    artista = item['artists'][0]['name']
                    artistas.add(artista)

                    results.append({
                        'name_track': item['name'],
                        'name_artist': artista,
                        'genre': genre,
                        'type': 'track',
                        'year': item['album']['release_date'][:4]
                    })
                    obtenidos += 1

                # Álbumes
                for item in albums:
                    if obtenidos >= max_por_anio:
                        break

                    artista = item['artists'][0]['name']
                    artistas.add(artista)

                    results.append({
                        'name_track': item['name'],
                        'name_artist': artista,
                        'genre': genre,
                        'type': 'album',
                        'year': item['release_date'][:4]
                    })
                    obtenidos += 1

                offset += 50  

            print(f" Total extraído {anio} ({genre}): {obtenidos}")

    return results, list(artistas)

In [5]:
genres = ['punk', 'indie', 'rock', 'pop' ]
anios = [2010, 2015, 2020, 2025]

results, artistas = extraer_datos_por_anios(
    sp=sp,
    genres=genres,
    anios=anios,
    max_por_anio=1000
)

print(len(results))
print(len(artistas))


 Extrayendo datos del año 2010
 Género: punk
 Total extraído 2010 (punk): 1000
 Género: indie
 Total extraído 2010 (indie): 1000
 Género: rock
 Total extraído 2010 (rock): 1000
 Género: pop
 Total extraído 2010 (pop): 1000

 Extrayendo datos del año 2015
 Género: punk
 Total extraído 2015 (punk): 1000
 Género: indie
 Total extraído 2015 (indie): 1000
 Género: rock
 Total extraído 2015 (rock): 1000
 Género: pop
 Total extraído 2015 (pop): 1000

 Extrayendo datos del año 2020
 Género: punk
 Total extraído 2020 (punk): 1000
 Género: indie
 Total extraído 2020 (indie): 1000
 Género: rock
 Total extraído 2020 (rock): 1000
 Género: pop
 Total extraído 2020 (pop): 1000

 Extrayendo datos del año 2025
 Género: punk
 Total extraído 2025 (punk): 1000
 Género: indie
 Total extraído 2025 (indie): 1000
 Género: rock
 Total extraído 2025 (rock): 1000
 Género: pop
 Total extraído 2025 (pop): 1000
16000
5734


In [6]:
# Extracción datos last.fm
api_key = "a03eaf07fb8aba070de6cbf312f88b25"
base_url = "http://ws.audioscrobbler.com/2.0/"

In [7]:
# Función extraer datos
import requests
import time

def obtener_info_artistas_lastfm(artistas, api_key, delay=0.25):

    base_url = "http://ws.audioscrobbler.com/2.0/"
    resultados = []

    for artista in artistas:
        params = {
            "method": "artist.getInfo",
            "artist": artista,
            "api_key": api_key,
            "format": "json"
        }

        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "artist" not in data:
                print(f"  {artista} No encontrado.")
                continue

            artist_data = data["artist"]

            resultados.append({
                "artista": artista,
                "biografia": artist_data.get("bio", {}).get("summary", ""),
                "Oyentes": int(artist_data.get("stats", {}).get("listeners", 0)),
                "N_Reproducciones": int(artist_data.get("stats", {}).get("playcount", 0)),
                "artistas_similares": [
                    a["name"] for a in artist_data.get("similar", {}).get("artist", [])
                ]
            })

            time.sleep(delay) 

        except Exception as e:
            print(f"Error con {artista}: {e}")

    return resultados

In [13]:
info_artistas = obtener_info_artistas_lastfm(artistas, api_key)

print(len(info_artistas))
info_artistas[0]

Error con The Runaways: Expecting value: line 2 column 1 (char 1)
  | Trippi | No encontrado.
Error con Galactic: Expecting value: line 2 column 1 (char 1)
   I Promised The World No encontrado.
  OmniMatraM No encontrado.
  CNQR+ No encontrado.
Error con Chartreuse: Expecting value: line 2 column 1 (char 1)
Error con Dudu Braga: Expecting value: line 2 column 1 (char 1)
  nyan.mp3 No encontrado.
  can’t be blue  No encontrado.
  Dreamthorne No encontrado.
5723


{'artista': 'Jacky Cheung',
 'biografia': 'Jacky Cheung Hok Yau (Traditional Chinese: 張學友; Simplified Chinese: 张学友; pinyin: Zhāng Xúeyǒu; born July 10, 1961 in Hong Kong with family roots in Tianjin) is a Hong Kong singer and film star from the mid-1980s to the present. Dubbed the "God of Songs" (歌神) by the Cantopop industry, Cheung is regarded by many as one of the finest male singers to have come out of Hong Kong, known for his rich baritone voice, wide range and exceptional interpretative powers. <a href="https://www.last.fm/music/+noredirect/Jacky+Cheung">Read more on Last.fm</a>',
 'Oyentes': 38091,
 'N_Reproducciones': 1210169,
 'artistas_similares': []}

In [8]:
import pandas as pd

In [9]:
df_results = pd.DataFrame(results)

In [10]:
df_results

,name_track,name_artist,genre,type,year
0,Perfect Weapon,Black Veil Brides,punk,track,2010
1,Whore,Get Scared,punk,track,2010
2,"With Ears to See, and Eyes to Hear",Sleeping With Sirens,punk,track,2010
3,Whiskey and Gin,The Killigans,punk,track,2010
4,A New Today,Endymion,punk,track,2010
...,...,...,...,...,...
15995,Pakikinggan Kita,Amiel Sol,pop,track,2025
15996,Kalawakan,Rob Deniel,pop,track,2025
15997,騙シ愛 - Damashiai,tuki.,pop,track,2025
15998,Зачарований,SAMCHUK,pop,track,2025


In [16]:
df_results.to_csv("spotify_base_datos_canciones_todas.csv", index=False)

In [11]:
df_artistas = pd.DataFrame(artistas)

In [12]:
df_artistas

,0
0,Jacky Cheung
1,AOA
2,Wallows
3,Modern Life Is War
4,Rea Garvey
...,...
5729,With Confidence
5730,Boatkeeper
5731,The Happy Fits
5732,Elles Bailey


In [17]:
df_artistas.to_csv("spotify_base_datos_artistas_todos.csv", index=False)

In [14]:
df_info_artistas = pd.DataFrame(info_artistas)

In [15]:
df_info_artistas

,artista,biografia,Oyentes,N_Reproducciones,artistas_similares
0,Jacky Cheung,Jacky Cheung Hok Yau (Traditional Chinese: 張學友...,38091,1210169,[]
1,AOA,There are at least four artists with this name...,588174,23341426,"[SISTAR, HELLO VENUS, EXID, 9MUSES, T-ARA]"
2,Wallows,Wallows is an American alternative rock band b...,1989343,120420748,"[Dayglow, Inhaler, Coin, Declan McKenna, HUNNY]"
3,Modern Life Is War,Modern Life is War is an American hardcore ban...,161191,6640083,"[Defeater, Killing The Dream, Shai Hulud, Vers..."
4,Rea Garvey,"Rea Garvey, born as Raymond Michael Garvey on ...",132094,2076798,"[Michael Patrick Kelly, Sunrise Avenue, Stanfo..."
...,...,...,...,...,...
5718,With Confidence,With Confidence were an Australian pop punk ba...,240146,6873585,"[State Champs, Between You & Me, Stand Atlanti..."
5719,Boatkeeper,"<a href=""https://www.last.fm/music/Boatkeeper...",9222,36707,"[Wolf and Willow, Andrew & Veda, Toby Johnson,..."
5720,The Happy Fits,The Happy Fits is an American band comprising ...,405938,13197124,"[Phoneboy, Saint Motel, Kevin Walkman, Jukebox..."
5721,Elles Bailey,Elles Bailey is a British singer from Bristol....,31757,298567,"[Danielle Nicole, Joanne Shaw Taylor, ZZ Ward,..."


In [18]:
df_info_artistas.to_csv("spotify_base_datos_info_artistas_todos.csv", index=False)